In [1]:
from selenium import webdriver as wd
from bs4 import BeautifulSoup as bs
import requests
import datetime, time
from datetime import date

import pandas as pd
import re
import os, sys

In [ ]:
# !pip install chromedriver_autoinstaller

import chromedriver_autoinstaller

driver_name = chromedriver_autoinstaller.install()  # Check if the current version of chromedriver exists
                                      # and if it doesn't exist, download it automatically,
                                      # then add chromedriver to path
time.sleep(1)

driver = wd.Chrome(driver_name)

In [ ]:
# driver.set_window_position(-10000,0) # hide windows
main_url = 'https://kr.investing.com/'
driver.get(main_url)
time.sleep(1)

In [ ]:
driver.find_element_by_css_selector('#userAccount > div > a.login.bold').click() 
time.sleep(1)

In [ ]:
driver.find_element_by_id('loginForm_password').clear()
driver.find_element_by_id('loginFormUser_email').send_keys('kangeun@naver.com')
time.sleep(1)

In [ ]:
driver.find_element_by_id('loginForm_password').clear()
driver.find_element_by_id('loginForm_password').send_keys(os.environ['investing_pass'])
time.sleep(1)

In [ ]:
driver.find_element_by_css_selector('#signup > a.newButton.orange').click()  
time.sleep(2)

In [ ]:
cpi_url = 'https://kr.investing.com/economic-calendar/cpi-733'

In [ ]:
driver.get(cpi_url)

In [ ]:
for _ in range(3):
    driver.find_element_by_id('showMoreHistory733').click()

In [ ]:
df = pd.read_html(driver.page_source)[0] # cpi dataframe

In [ ]:
df.columns=  ['annouce_date', 'time', 'real_rate', 'anticipated_rate', 'prev_rate', 'none']

In [ ]:
for i, date in enumerate(df['annouce_date']):
    y_m_d = re.findall(r'\d+', date)
    time = datetime.date(int(y_m_d[0]), int(y_m_d[1]), int(y_m_d[2]))
    df.loc[i,'annouce_date'] = time

In [ ]:
for i, date in enumerate(df['time']):
    time = datetime.time(int(date.split(':')[0]), int(date.split(':')[1]))
    df.loc[i,'time'] = time

In [ ]:
df

In [ ]:
df.to_csv('data/cpi_index_20220610.csv')

In [ ]:
real_time_future_url = 'https://kr.investing.com/indices/indices-futures'

In [ ]:
driver.get(real_time_future_url)

In [ ]:
'''
driver.find_element_by_css_selector('#widgetFieldDateRange').click()

driver.find_element_by_id('startDate').clear()
driver.find_element_by_id('startDate').send_keys('2022/04/20')

driver.find_element_by_id('endDate').clear()
driver.find_element_by_id('endDate').send_keys('2022/05/19')

driver.find_element_by_css_selector('#applyBtn').click()

data_download_sec = '#column-content > div.float_lang_base_2.downloadDataWrap > div > a'
driver.find_element_by_css_selector(data_download_sec).click()
'''

In [ ]:
dji_url = 'https://kr.investing.com/indices/us-30-historical-data'
nas_url = 'https://kr.investing.com/indices/nasdaq-composite-historical-data'
spx_url = 'https://kr.investing.com/indices/us-spx-500-historical-data'
kospi_url = 'https://kr.investing.com/indices/kospi-historical-data'
kosdaq_url = 'https://kr.investing.com/indices/kosdaq-historical-data'
wti_url = 'https://kr.investing.com/commodities/crude-oil-historical-data'
urls = [dji_url, nas_url, spx_url, kospi_url, kosdaq_url, wti_url]

In [ ]:
# 검색하고자 하는 일정 기간 정의
start_date = ['2021/12/01', '2022/02/01', '2022/04/01']
end_date = ['2022/01/31', '2022/03/31', '2022/05/31']

In [ ]:
for url in urls:
    driver.get(url)
    time.sleep(1)
    get_index_historical_data(start_date, end_date)

In [ ]:
import re

In [ ]:
real_time_future_url = 'https://kr.investing.com/indices/indices-futures'
cpi_url = 'https://kr.investing.com/economic-calendar/cpi-733'
url_list = [ real_time_future_url, cpi_url]

In [ ]:
url_list

In [ ]:
url1 = url_list[0]
res = requests.get(url1)
soup = bs(res.text, 'html.parser')

In [ ]:
url1 = 'https://kr.investing.com/commodities/crude-oil-historical-data'
res = requests.get(url1)
soup = bs(res.text, 'html.parser')
wti = soup.select('#results_box tr')

wti_date_value = {}
for i, tr in enumerate(wti):
    if (i == 0) | (i == len(wti)-1) : continue
    tr_value = tr.select('td')
    wti_date_value[tr_value[0].text] = tr_value[1].text

In [ ]:
from datetime import date
import re
# date_list = []
# for i, (date1, val) in enumerate(wti_date_value.items()):
#     y_m_d = re.findall(r'\d+', date1)
#     time = datetime.date(int(y_m_d[0]), int(y_m_d[1]), int(y_m_d[2]))
#     date_list.append(time)
    
data_dict = {}
date_list = []
wti_list = []
for i, (date1, val) in enumerate(wti_date_value.items()):
    y_m_d = re.findall(r'\d+', date1)
    time = datetime.date(int(y_m_d[0]), int(y_m_d[1]), int(y_m_d[2]))
    date_list.append(time)
    wti_list.append(val)

data_dict['date'] = date_list
data_dict['wti'] = wti_list

df_wti = pd.DataFrame(data_dict) 
df_wti.tail()

In [ ]:
date_list

In [ ]:
nas_url = 'https://www.investing.com/indices/nasdaq-composite-chart'
res = requests.get(nas_url)
soup = bs(res.text, 'html.parser')
nas = soup.select('#last_last')[0].text
nas = float(nas.replace(",", ""))
nas

In [ ]:
dow_url = 'https://www.investing.com/indices/us-30'
res = requests.get(dow_url)
soup = bs(res.text, 'html.parser')
select_path = '#__next > div.desktop\:relative.desktop\:bg-background-default > div > div > div.grid.gap-4.tablet\:gap-6.grid-cols-4.tablet\:grid-cols-8.desktop\:grid-cols-12.grid-container--fixed-desktop.general-layout_main__3tg3t > main > div > div.instrument-header_instrument-header__1SRl8.mb-5.bg-background-surface.tablet\:grid.tablet\:grid-cols-2 > div:nth-child(2) > div.instrument-price_instrument-price__3uw25.flex.items-end.flex-wrap.font-bold > span'
dow = soup.select('span.text-2xl')[0].text
dow = float(dow.replace(",", ""))
dow

In [ ]:
snp_url = 'https://www.investing.com/indices/us-spx-500'
res = requests.get(snp_url)
soup = bs(res.text, 'html.parser')
snp = soup.select('span.text-2xl')[0].text
snp = float(snp.replace(",", ""))
snp

삼성전자 투자자별

In [ ]:
path = 'body > table:nth-child(4) > tbody > tr > td > p:nth-child(6) > table > tbody > tr:nth-child(3) > td:nth-child(2)'

In [2]:
ss_url = 'https://vip.mk.co.kr/newSt/price/trade.php?stCode=005930'
res = requests.get(ss_url)
soup = bs(res.text, 'html.parser')
tables = soup.select('table')

In [3]:
df = pd.read_html('https://vip.mk.co.kr/newSt/price/trade.php?stCode=005930')

In [26]:
df = pd.read_html('https://open.shinhaninvest.com/goodicyber/mk/1206.jsp?code=005930')
len(df)

8

In [31]:
df[7]

,0,1,2,3,4,5,6,7,8,9,10
0,날짜,개인,외국인계,기관,기관,기관,기관,기관,기관,기관,기타
1,날짜,개인,외국인계,기관계,증권,투신,은행,종금,보험,기금,기타
2,2022/06/20,2122832,-4421258,2247165,2348239,-29504,3399,26299,-5224,-61619,38068
3,2022/06/17,6494723,-7355651,612667,1764379,-453608,54668,24249,-114845,-473784,209737
4,2022/06/16,685287,-552199,-223863,209601,14150,-29652,-48134,-14528,-346840,71295
5,2022/06/15,3730369,-4324395,526298,617784,-86095,2575,14655,-43153,-621588,61968
6,2022/06/14,588203,-1154038,390583,623213,71902,-50463,3000,-69691,-161545,158836
7,2022/06/13,3222323,-2813392,-511162,234735,-50997,-71790,4787,-210523,-130427,79304
8,2022/06/10,10428214,-9306686,-1133682,-925727,-256036,-11803,9920,29089,135388,-31101
9,2022/06/09,5858885,-6429047,448925,3507497,-986680,-17311,-467,-366707,-1467577,97635


In [ ]:
# 현대차 005830, 네이버 035420, KT&G 033780, 

In [40]:
df = pd.read_html('https://open.shinhaninvest.com/goodicyber/mk/1206.jsp?code=005930')
df[7].to_csv('data/samsung_invester_from-420.csv')

In [41]:
df = pd.read_html('https://open.shinhaninvest.com/goodicyber/mk/1206.jsp?code=005830')
df[7].to_csv('data/hyuncar_invester_from-420.csv')

In [42]:
df = pd.read_html('https://open.shinhaninvest.com/goodicyber/mk/1206.jsp?code=035420')
df[7].to_csv('data/naver_invester_from-420.csv')

In [43]:
df = pd.read_html('https://open.shinhaninvest.com/goodicyber/mk/1206.jsp?code=033780')
df[7].to_csv('data/ktng_invester_from-420.csv')

In [61]:
df = pd.read_html('https://vip.mk.co.kr/newSt/price/minprice.php?type=10&stCode=005930&p_page=1')
df1 = df[10]

In [62]:
df = pd.read_html('https://vip.mk.co.kr/newSt/price/minprice.php?type=10&stCode=005930&p_page=2')
df1 = df1.append(df[10])

In [63]:
df = pd.read_html('https://vip.mk.co.kr/newSt/price/minprice.php?type=10&stCode=005930&p_page=3')
df1 = df1.append(df[10])

In [64]:
df1

,시간,현재가,등락,등락률(%),체결량,거래량
0,06/20 15:50,58700,"▼ 1,100",-1.84,6340,33882616
1,06/20 15:40,58700,"▼ 1,100",-1.84,31386,33876276
2,06/20 15:30,58700,"▼ 1,100",-1.84,737827,33844890
3,06/20 15:10,58700,"▼ 1,100",-1.84,608550,33107063
4,06/20 15:00,58500,"▼ 1,300",-2.17,1193691,32498513
5,06/20 14:50,58400,"▼ 1,400",-2.34,750173,31304822
6,06/20 14:40,58300,"▼ 1,500",-2.51,406133,30554649
7,06/20 14:30,58200,"▼ 1,600",-2.68,660258,30148516
8,06/20 14:20,58200,"▼ 1,600",-2.68,503835,29488258
9,06/20 14:10,58200,"▼ 1,600",-2.68,419541,28984423


In [78]:
for i in range(3):
    print(i)
    str1 = str(i+1)
    url = 'https://vip.mk.co.kr/newSt/price/minprice.php?type=10&stCode=005930&p_page='+str1
    print(url)
    df = pd.read_html(url)
    if i == 0 :
        df1 = df[10]
    else:
        df1 = df1.append(df[10])

0
https://vip.mk.co.kr/newSt/price/minprice.php?type=10&stCode=005930&p_page=1
1
https://vip.mk.co.kr/newSt/price/minprice.php?type=10&stCode=005930&p_page=2
2
https://vip.mk.co.kr/newSt/price/minprice.php?type=10&stCode=005930&p_page=3


In [80]:
df1.to_csv('data/samsung_10min_price_from-420.csv')

In [81]:
for i in range(3):
    print(i)
    str1 = str(i+1)
    url = 'https://vip.mk.co.kr/newSt/price/minprice.php?type=10&stCode=005830&p_page='+str1
    print(url)
    df = pd.read_html(url)
    if i == 0 :
        df1 = df[10]
    else:
        df1 = df1.append(df[10])

0
https://vip.mk.co.kr/newSt/price/minprice.php?type=10&stCode=005830&p_page=1
1
https://vip.mk.co.kr/newSt/price/minprice.php?type=10&stCode=005830&p_page=2
2
https://vip.mk.co.kr/newSt/price/minprice.php?type=10&stCode=005830&p_page=3


In [83]:
df1.to_csv('data/hyuncar_10min_price_from-420.csv')

In [84]:
for i in range(3):
    print(i)
    str1 = str(i+1)
    url = 'https://vip.mk.co.kr/newSt/price/minprice.php?type=10&stCode=035420&p_page='+str1
    print(url)
    df = pd.read_html(url)
    if i == 0 :
        df1 = df[10]
    else:
        df1 = df1.append(df[10])

0
https://vip.mk.co.kr/newSt/price/minprice.php?type=10&stCode=035420&p_page=1
1
https://vip.mk.co.kr/newSt/price/minprice.php?type=10&stCode=035420&p_page=2
2
https://vip.mk.co.kr/newSt/price/minprice.php?type=10&stCode=035420&p_page=3


In [85]:
df1.to_csv('data/naver_10min_price_from-420.csv')

In [86]:
for i in range(3):
    print(i)
    str1 = str(i+1)
    url = 'https://vip.mk.co.kr/newSt/price/minprice.php?type=10&stCode=033780&p_page='+str1
    print(url)
    df = pd.read_html(url)
    if i == 0 :
        df1 = df[10]
    else:
        df1 = df1.append(df[10])

0
https://vip.mk.co.kr/newSt/price/minprice.php?type=10&stCode=033780&p_page=1
1
https://vip.mk.co.kr/newSt/price/minprice.php?type=10&stCode=033780&p_page=2
2
https://vip.mk.co.kr/newSt/price/minprice.php?type=10&stCode=033780&p_page=3


In [87]:
df1.to_csv('data/ktng_10min_price_from-420.csv')

In [88]:
df1

,시간,현재가,등락,등락률(%),체결량,거래량
0,06/20 15:50,82500,▼ 900,-1.08,31,246682
1,06/20 15:40,82500,▼ 900,-1.08,45,246651
2,06/20 15:30,82500,▼ 900,-1.08,8234,246606
3,06/20 15:10,82500,▼ 900,-1.08,7609,238372
4,06/20 15:00,82400,"▼ 1,000",-1.20,6358,230763
5,06/20 14:50,82400,"▼ 1,000",-1.20,7473,224405
6,06/20 14:40,82200,"▼ 1,200",-1.44,5089,216932
7,06/20 14:30,82500,▼ 900,-1.08,13810,211843
8,06/20 14:20,82200,"▼ 1,200",-1.44,5655,198033
9,06/20 14:10,82300,"▼ 1,100",-1.32,6188,192378


In [ ]:
df1.to_csv('data/samsung_10min_price_from-420.csv')

In [65]:
str1 = str(0+1)
str1


'1'

In [48]:
for i in range(df):
    i

TypeError: 'list' object cannot be interpreted as an integer

In [60]:
df1

,시간,현재가,등락,등락률(%),체결량,거래량
0,06/20 15:50,58700,"▼ 1,100",-1.84,6340,33882616
1,06/20 15:40,58700,"▼ 1,100",-1.84,31386,33876276
2,06/20 15:30,58700,"▼ 1,100",-1.84,737827,33844890
3,06/20 15:10,58700,"▼ 1,100",-1.84,608550,33107063
4,06/20 15:00,58500,"▼ 1,300",-2.17,1193691,32498513
5,06/20 14:50,58400,"▼ 1,400",-2.34,750173,31304822
6,06/20 14:40,58300,"▼ 1,500",-2.51,406133,30554649
7,06/20 14:30,58200,"▼ 1,600",-2.68,660258,30148516
8,06/20 14:20,58200,"▼ 1,600",-2.68,503835,29488258
9,06/20 14:10,58200,"▼ 1,600",-2.68,419541,28984423


In [57]:
for i in range(len(df)):
    print(i, df[i])

0                                                   시간      현재가  \
                                                 종목명      현재가   
0                                                코스피  2391.03   
1                                                코스닥   769.92   
2  삼성전자(005930)  코스피 200전기,전자  액면가 100원  06.20 15...      NaN   

                                                  등락  등락률(%)  \
                                                 전일비     등락률   
0                                             ▼ 49.9  -2.04%   
1                                            ▼ 28.77  -3.60%   
2  ID찾기 │ 회원가입 │ 서비스신청 아이디저장  06.20 15:59 실시간신청  ...     NaN   

                 체결량                거래량  
  Unnamed: 4_level_1 Unnamed: 5_level_1  
0                NaN                NaN  
1                NaN                NaN  
2                NaN                NaN  
1                                       0  \
0  삼성전자(005930)  코스피 200전기,전자  액면가 100원   
1  삼성전자(005930)  코스피 200전기,전자  액면가 100원   

         

In [ ]:
df = pd.read_html('https://vip.mk.co.kr/newSt/price/trade.php?stCode=005930', header=0)[0]
df.head()

In [ ]:
tables[0]

In [ ]:
ss_url = 'https://vip.mk.co.kr/newSt/price/trade.php?stCode=005930'
res = requests.get(ss_url)
soup = bs(res.text, 'html.parser')
tables = soup.select('tbody')

# 현재 페이지에서 table 태그 모두 선택하기
# tables = soup.select('table')

# 하나의 테이블 태그 선택하기
table = tables[0]

In [ ]:
tables[0]

In [ ]:


# 테이블 html 정보를 문자열로 변경하기
table_html = str(table)

# 판다스의 read_html 로 테이블 정보 읽기
table_df_list = pd.read_html(table_html, header=0, encoding='utf-8')

# 데이터프레임 선택하기
table_df = table_df_list[0]

In [ ]:
table_df.head()

In [ ]:
df = pd.read_html('<table>' + str(table) + '</table>', encoding='euc-kr')[0]

In [ ]:
df.head()

In [ ]:
ss_url = 'https://vip.mk.co.kr/newSt/price/trade.php?stCode=005930'
res = requests.get(ss_url)
soup = bs(res.text, 'html.parser')
tables = soup.select('tbody')

# 현재 페이지에서 table 태그 모두 선택하기
# tables = soup.select('table')

# 하나의 테이블 태그 선택하기
table = tables[0]

In [ ]:
ss_url = 'https://vip.mk.co.kr/newSt/price/trade.php?stCode=005930'

In [ ]:
res = requests.get(ss_url)

In [ ]:
print(res.encoding)

In [ ]:
res.raise_for_status()

In [ ]:
res.encoding = 'euc-kr'

In [ ]:
res.text

In [ ]:
soup = bs(res.text, 'html.parser')

In [ ]:
print(soup.prettify())

In [ ]:
tables = soup.select('table.content1')

# 현재 페이지에서 table 태그 모두 선택하기
# tables = soup.select('table')

In [ ]:
tables

In [ ]:
# 하나의 테이블 태그 선택하기
table = tables[0]

In [ ]:
df = pd.read_html('<table>' + str(table) + '</table>')
# df = pd.read_html(str(table))

In [ ]:
df

In [ ]:
from urllib.parse import urlparse
from urllib.parse import parse_qs

In [ ]:
url = 'https://open.shinhaninvest.com/goodicyber/mk/1206.jsp?code=005930'

In [ ]:
parsed_url = urlparse(url)

In [ ]:
captured_value = parse_qs(parsed_url.query)['code'][0]

In [ ]:
print(captured_value)

In [ ]:
payload = {"code":"005380"}
page = requests.get("https://open.shinhaninvest.com/goodicyber/mk/1206.jsp?", params=payload).text

print(page)

In [ ]:
driver.quit()
driver.close()